# Modelos e Estratégias de Trading
## Ajustando um modelo VAR à PETR4

Uirá Caiado. 05 de Setembro, 2016

**Resumo**

*Uma das principais necessidades quando falamos trading é realizar algum tipo de previsão sobre o estado futuro do instrumento operado para que possamos posicionar nossas ofertas. Como um dos objetivos da análise de séries temporais multivariadas é a realização de forecasts, nesta atividade vou implementar um modelo desta classe chamado Vetor Auto Regressivo (VAR). Vou ajustar o modelo aos retornos e informações de book da PETR4 e analisar a performance realizando um back-test em uma parte dos dados coletados que ainda não tenha sido utilizada.*

## 1. Introdução

Nesta sessão vou ...


### 1.1. Vetor Auto Regressivo (VAR)

bla bla

Pressupostos



### 1.2. O Problema
bla bla

## 2. Implementando o Modelo

Nesta sessão detalharemos e implementaremos ...

### 2.1. Estimando os Parâmetros

bla bla

In [1]:
import pandas as pd
df = pd.read_excel('data/VAR3_inputs_e_outputs.xlsx', header=None)
l_index = pd.date_range(start= '01/01/2014', end='01/12/2016')
df.index = l_index[:df.shape[0]]

In [2]:
from statsmodels.tsa.api import VAR
import statsmodels.tsa.vector_ar.util as util
model = VAR(df)
results = model.fit(3)
aux = util.get_var_endog(df.values, 3)

In [82]:
import var_model.vector_autoregression as var
reload(var)
self = var.VectorAutoregression(df)
self.fit(3)

In [73]:
# calcula erros para implementacao do statsmodel
print "error in Z: {:0.8f}".format(sum(sum(abs(self.na_Z.T - aux))))
print "error in A: {:0.8f}".format(sum(sum(sum(abs(self.na_A - results.coefs)))))
print "error in B: {:0.8f}".format(sum(sum(abs(self.na_betahat - results.params.values))))
print "error in residuals: {:0.8f}".format(sum(sum(abs(self.na_U.T - results.resid.values))))
print "error in Sigma U: {:0.8f}".format(sum(sum(abs(self.na_Sigma - results.sigma_u.values))))

error in Z: 0.00000000
error in A: 0.00000000
error in B: 0.00000000
error in residuals: 0.00000000
error in Sigma U: 0.00000000


### 2.2. Forecast

Segundo notas de aula, para se afirmar o que ocorrerá no futuro $\left (  y_1, ..., y_k \right )$, tendo um processo $VAR(p)$ ajustado a um conjunto de dados $\Omega_t = \{ y_s \mid s \leq t \}$ para um horizonte de tempo $h$, precisamos determinar qual o *forecast* ótimo determinando aquele que minimiza uma função custo associada à seus erros (quadráticos médios - MSE na sigla em inglês). O preditor que minimiza estes erros é a esperança condicional (Lutkepohl, p. 33)

$$\mathbf{E}\left [ y_{t+h} \right ] := \mathbf{E}\left [ y_{t+h} \mid  \Omega_t \right ] = \mathbf{E}\left [ y_{t+h} \mid  \{ y_s \mid s \leq t \} \right ]$$

Lutkepohl ainda demonstra que a otimização da esperança condicional impica que:

$$
\mathbf{E}\left [ y_{t+1} \right ] = \upsilon + A_1 y_t + ... + A_p y_{t-p + 1} \\
\mathbf{E}\left [ y_{t+2} \right ] = \upsilon + A_1 \mathbf{E}\left [ y_{t+1} \right ] + A_2 y_t + ... + A_p y_{t-p + 2} \\
\vdots
$$


### 2.3. Intervalo de Confiança

Segundo Lutkepohl (p. 38), o erro de previsão (e consequentemente, o intervalo de confiança) pode ser obtido através da matriz de covariância dos erros (ou, da sigla em inglês, MSE matrix). Porém, quando se deseja realizar *forecasts* para mais de um período, é necessário definir também a matriz de coeficientes de Média Móvel (MA), que para um VAR(2) fica:

$$
\phi_1 = A_1 \\
\phi_2 = \phi_1 A_1 + A2 \\
\phi_3 = \phi_2 A_1 + \phi_1 A2 \\
\vdots \\
\phi_i = \phi_{i-1} A_1 + \phi_{i-2} A_2
$$

Assim, a matriz MSE de *forecast* são obtidas recursivamente aplicando

$$
\Gamma_y(0) = \Sigma_y = \sum_{i=0}^{\infty} \phi_i \Sigma_u \phi_i^{'}
$$

Sendo que:
$$
\Sigma_y (1) = \Sigma_u \\
\Sigma_y (2) = \Sigma_u + \phi_{1} \Sigma_u \phi_{1}' \\
\Sigma_y (3) = \Sigma_y (2) + \phi_{2} \Sigma_u \phi_{2}' \\
\vdots
$$

O $h$ em $\Sigma_y (h)$ se refere a quantos passos para frente se aplica a função de forecast. Como o modelo de VAR assume que os erros $u_t \sim N \left( 0, \, \Sigma_y(h) \right)$, podemos assumir que o erro de *forecast* também é normalmente distribuído. Com este pressuposto, podemos definir um [intervalo de confiança](https://en.wikipedia.org/wiki/Confidence_interval) na forma: 
$$\left[ y_{k, \, t} (h) - z_{(\alpha / 2)} \sigma_k (h), \, \, \, \, \, y_{k, \, t} (h) + z_{(\alpha / 2)} \sigma_k (h) \right]$$

Onde $\sigma(h)$ é a raiz quadrada do ** *k-ésimo* elemento da diagonal** $\Sigma_y(h)$ de maneira que

TODO: escrever exemplo da página 38 para não me perder depois

In [171]:
import var_model.vector_autoregression as var
reload(var)
self = var.VectorAutoregression(df)
self.fit(3)

In [172]:
self.forecast(df[-5:].values, 3)

(array([-0.00182718, -0.00110312, -0.00133409, -0.00109055, -0.0031575 ]),
 array([ 0.01471869,  0.02090863,  0.02205742,  0.02227275,  0.04007668]),
 array([-0.01837305, -0.02311488, -0.0247256 , -0.02445384, -0.04639168]))

In [178]:
sum(sum(abs(self.na_sigma_y - covs[0])))

7.329567326133623e-05

In [135]:
na_forecast, na_min, na_max = results.forecast_interval(df[-3:].values, 3)

In [143]:
na_min[-1]

array([-0.0304958 , -0.03620013, -0.03791632, -0.03508742, -0.04711637])

In [141]:
na_forecast[-1]

array([-0.00182718, -0.00110312, -0.00133409, -0.00109055, -0.0031575 ])

In [140]:
na_max[-1]

array([ 0.02684144,  0.03399389,  0.03524813,  0.03290633,  0.04080137])

In [ ]:
self.

In [149]:
covs = results.forecast_cov(5)

In [150]:
inds = np.arange(results.neqs)

In [151]:
inds

array([0, 1, 2, 3, 4])

In [152]:
na_vars = results._forecast_vars(5)

In [155]:
covs[:, inds, inds]

array([[ 0.00021105,  0.00030629,  0.0003343 ,  0.00028909,  0.00049847],
       [ 0.00021259,  0.00031079,  0.00032977,  0.00028874,  0.00049291],
       [ 0.00021395,  0.00032066,  0.00034837,  0.00030087,  0.00050303],
       [ 0.0002147 ,  0.00031908,  0.0003448 ,  0.00029547,  0.00049958],
       [ 0.00021487,  0.00031924,  0.00034478,  0.00029543,  0.00049882]])

array([[  2.11050519e-04,   7.39080209e-05,   6.00288811e-05,
          8.30239376e-05,   6.94379229e-05],
       [  7.39080209e-05,   3.06290537e-04,   1.66864395e-04,
          1.79644941e-04,   1.26058532e-04],
       [  6.00288811e-05,   1.66864395e-04,   3.34295101e-04,
          2.76139118e-04,   1.35934416e-04],
       [  8.30239376e-05,   1.79644941e-04,   2.76139118e-04,
          2.89093225e-04,   1.40798750e-04],
       [  6.94379229e-05,   1.26058532e-04,   1.35934416e-04,
          1.40798750e-04,   4.98471180e-04]])

In [157]:
mse = results.mse(steps)

In [159]:
results._omega_forc_cov(steps)

array([[[ 0.00312315,  0.0010937 ,  0.00088831,  0.0012286 ,  0.00102755],
        [ 0.0010937 ,  0.00453252,  0.00246928,  0.00265841,  0.00186543],
        [ 0.00088831,  0.00246928,  0.00494694,  0.00408634,  0.00201157],
        [ 0.0012286 ,  0.00265841,  0.00408634,  0.00427804,  0.00208356],
        [ 0.00102755,  0.00186543,  0.00201157,  0.00208356,  0.00737644]],

       [[ 0.00223352,  0.00084324,  0.00065742,  0.0009067 ,  0.00067963],
        [ 0.00084324,  0.00337521,  0.0018222 ,  0.00199397,  0.00131364],
        [ 0.00065742,  0.0018222 ,  0.00345322,  0.00288452,  0.00139159],
        [ 0.0009067 ,  0.00199397,  0.00288452,  0.0030486 ,  0.00142816],
        [ 0.00067963,  0.00131364,  0.00139159,  0.00142816,  0.0051463 ]],

       [[ 0.00139082,  0.00057221,  0.0004765 ,  0.00059665,  0.00047901],
        [ 0.00057221,  0.00225028,  0.00123733,  0.0013139 ,  0.00086517],
        [ 0.0004765 ,  0.00123733,  0.00234956,  0.00198181,  0.00105008],
        [ 0.00059665,

In [161]:
phis = results.ma_rep(steps)

In [170]:
print 'Phis erros: {:.7f}'.format(sum(sum(sum(abs(phis - self._estimate_ma_phis(steps))))))

Phis erros: 0.0000000


array([[[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           1.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   1.00000000e+00]],

       [[ -4.33740266e-02,   1.28951234e-01,  -1.27478901e-01,
           4.96482766e-02,  -7.12502594e-02],
        [  5.32105776e-03,   1.78780028e-01,   3.75889103e-03,
           3.25092218e-02,  -7.22829526e-02],
        [  8.24173822e-03,   1.15179836e-01,  -5.07079197e-02,
           7.21155964e-03,  -4.19460385e-02],
        [  3.58036613e-02,   1.66959303e-01,   4.86540416e-02,
          -1.17872920e-01,  -4.98642496e-02],
        [ -6.4091

In [158]:
mse

array([[[  1.95196959e-04,   6.83562446e-05,   5.55196693e-05,
           7.67873977e-05,   6.42219286e-05],
        [  6.83562446e-05,   2.83282797e-04,   1.54329980e-04,
           1.66150485e-04,   1.16589346e-04],
        [  5.55196693e-05,   1.54329980e-04,   3.09183732e-04,
           2.55396274e-04,   1.25723380e-04],
        [  7.67873977e-05,   1.66150485e-04,   2.55396274e-04,
           2.67377302e-04,   1.30222318e-04],
        [  6.42219286e-05,   1.16589346e-04,   1.25723380e-04,
           1.30222318e-04,   4.61027336e-04]],

       [[  2.01255130e-04,   7.25609263e-05,   5.91637177e-05,
           8.06808895e-05,   6.44833291e-05],
        [  7.25609263e-05,   2.93655403e-04,   1.59114257e-04,
           1.72831991e-04,   1.18740066e-04],
        [  5.91637177e-05,   1.59114257e-04,   3.12239270e-04,
           2.59286437e-04,   1.25691184e-04],
        [  8.06808895e-05,   1.72831991e-04,   2.59286437e-04,
           2.73268276e-04,   1.29005927e-04],
        [  6.4483

### 2.4. Selecionando a Ordem do VAR

bla bla

## 3. Aplicando o modelo VAR

Nesta seção ...


## 4. Conclusão

bla bla




## 5. Últimas Considerações

bla bla


*Style notebook and change matplotlib defaults*

In [9]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

In [6]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set2", 10))